# Import the Necessary Libraries

In [285]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine

In [286]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 1.4.39
Running PyMongo Version: 4.6.3


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working 


In [287]:
mysql_args = {
    "uid" : "root",
    "pwd" : "Passw0rd123",
    "hostname" : "localhost",
    "dbname" : "adventureworks"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "",
    "password" : "password",
    "cluster_name" : "cluster_name",
    "cluster_subnet" : "xxxxx",
    "cluster_location" : "local",
    "db_name" : "adventureworks"
}

#### Declare & Assign Connection Variables for the MySQL Server & Databases with which You'll be Working 

In [288]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Passw0rd123"

src_dbname = "adventureworks"
dst_dbname = "adventureworks_dw"

#### Create the New Data Warehouse database, and to Use it, Switch the Connection Context.

In [289]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(f"CREATE DATABASE IF NOT EXISTS`{dst_dbname}`;")
connection.execute(f"USE {dst_dbname};")

connection.close()

#### Define Functions for Getting Data From and Setting Data Into Databases

In [290]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

#### Populate MongoDB with Source Data


In [291]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"customers" : 'adventureworks_customer.json',
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)         

### 1.0. Create and Populate the New Dimension Tables
#### 1.1. Extract Data from the Source MongoDB Collections Into DataFrames

In [292]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "customers"

df_customers = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_customers.head(2)

,CustomerID,TerritoryID,AccountNumber,CustomerType,ModifiedDate
0,1,1,AW00000001,S,2004-10-13 11:15:07
1,2,1,AW00000002,S,2004-10-13 11:15:07


In [293]:
adventureworks_sales = "SELECT * FROM adventureworks.salesorderdetail;"
df_sales = get_sql_dataframe(adventureworks_sales, **mysql_args)
df_sales.head(2)

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,"b'm\xc9\x07\xb2\xe6\xd9+@\x84p,\xc1v\xc4""\x83'",2001-07-01
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,"b'\r`\xbbzw\x1e\xbeA\x9f\xe5\xb9\x14,\xfc\x08\...",2001-07-01


In [294]:
adventureworks_sales_header = "SELECT * FROM adventureworks.salesorderheader;"
df_sales_header = get_sql_dataframe(adventureworks_sales_header, **mysql_args)
df_sales_header.head(2)

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43659,1,2001-07-01,2001-07-13,2001-07-08,5,b'\x00',SO43659,PO522145787,10-4020-000676,...,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None,b'!S\xb6y\xca9\x15A\x9c\xba\x8f\xe0\x90>\x12\xe6',2001-07-08
1,43660,1,2001-07-01,2001-07-13,2001-07-08,5,b'\x00',SO43660,PO18850127500,10-4020-000117,...,5618.0,115213Vi29411,NaN,1553.1035,124.2483,38.8276,1716.1794,None,"b'-\xc4\x8ds;\xd0\xa1H\x98""\xf9Zg\xeas\x89'",2001-07-08


In [295]:
query = "SELECT * FROM adventureworks_dw.dim_date;"
df_dim_date = get_sql_dataframe(query, **mysql_args)
df_dim_date.head(2)

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
1,20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


#### Perform Any Necessary Transformations

In [296]:
df_customers.rename(columns={"CustomerID":"CustomerKey"}, inplace=True)
drop_cols = ['ModifiedDate']
df_customers.drop(drop_cols, axis=1, inplace=True)
df_customers.head(2)

,CustomerKey,TerritoryID,AccountNumber,CustomerType
0,1,1,AW00000001,S
1,2,1,AW00000002,S


In [297]:
df_sales.rename(columns={"SalesOrderDetailID":"SalesOrderKey"}, inplace=True)
drop_cols = ['rowguid','ModifiedDate']
df_sales.drop(drop_cols, axis=1, inplace=True)
df_sales.head(2)

,SalesOrderID,SalesOrderKey,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982


In [298]:
df_sales_header.insert(0, "sales_order_key", range(1, df_sales_header.shape[0]+1))
drop_cols = ['rowguid','ModifiedDate','OnlineOrderFlag']
df_sales_header.drop(drop_cols, axis=1, inplace=True)
df_sales_header['OrderDate'] = df_sales_header['OrderDate'].dt.date
df_sales_header.head()

,sales_order_key,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment
0,1,43659,1,2001-07-01,2001-07-13,2001-07-08,5,SO43659,PO522145787,10-4020-000676,...,985,5,16281.0,105041Vi84182,NaN,24643.9362,1971.5149,616.0984,27231.5495,None
1,2,43660,1,2001-07-01,2001-07-13,2001-07-08,5,SO43660,PO18850127500,10-4020-000117,...,921,5,5618.0,115213Vi29411,NaN,1553.1035,124.2483,38.8276,1716.1794,None
2,3,43661,1,2001-07-01,2001-07-13,2001-07-08,5,SO43661,PO18473189620,10-4020-000442,...,517,5,1346.0,85274Vi6854,4.0,39422.1198,3153.7696,985.5530,43561.4424,None
3,4,43662,1,2001-07-01,2001-07-13,2001-07-08,5,SO43662,PO18444174044,10-4020-000227,...,482,5,10456.0,125295Vi53935,4.0,34689.5578,2775.1646,867.2389,38331.9613,None
4,5,43663,1,2001-07-01,2001-07-13,2001-07-08,5,SO43663,PO18009186470,10-4020-000510,...,1073,5,4322.0,45303Vi22691,NaN,503.3507,40.2681,12.5838,556.2026,None


#### 1.3. Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables

Here we will call our **set_dataframe( )** function to create each dimension table. This function expects a number of parameters including the usual connection information (e.g., user_id, password, MySQL server name and database), the *table_name* we need to assign to the table, the *pandas DataFrame* we crafted to define & populate the table, the *name* of the column we wish to designate as the *primary_key* column, and finally, the database operation (insert or update). 

In [299]:
db_operation = "insert"

dim_tables = [('dim_customers', df_customers, 'CustomerKey'),
          ('dim_sales', df_sales, 'SalesOrderKey'),
          ('dim_date', df_dim_date, 'date_key')]

In [300]:
for table_name, dataframe, primary_key in dim_tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [301]:
db_operation = "insert"

fact_table = [('dim_sales_header', df_sales_header, 'sales_order_key')]

In [302]:
for table_name, dataframe, primary_key in fact_table:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

####  Author one or more SQL queries (SELECT statements) to demonstrate proper functionality

In [308]:
sql_orders_2000 = """
    SELECT
        adventureworks_dw.dim_customers.CustomerKey,
        SUM(adventureworks_dw.dim_sales_header.TotalDue) AS TotalSalesAmount,
        SUM(adventureworks_dw.dim_sales.OrderQty) AS TotalQuantitySold
    FROM
        adventureworks_dw.dim_sales_header
    LEFT JOIN
        adventureworks_dw.dim_sales ON dim_sales_header.sales_order_key =dim_sales.SalesOrderKey
    LEFT JOIN
        adventureworks_dw.dim_customers ON dim_sales_header.CustomerID = dim_customers.CustomerKey
    LEFT JOIN
        adventureworks_dw.dim_date ON dim_sales_header.OrderDate = dim_date.full_date
    WHERE
        dim_date.calendar_year = 2000
    GROUP BY
        dim_customers.CustomerKey; 
"""
df_orders_2000 = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_orders_2000)
df_orders_2000.head()

,CustomerKey,TotalSalesAmount,TotalQuantitySold
